# Pseudo-code for project 2

OBJECTIVE: Find parameters of shots that lead to goals using exhaustive parameter scan

In [6]:
import numpy as np
import pandas as pd

import ode

import matplotlib.pyplot as plt
%matplotlib notebook

Should be at least 3 pages long

First, define a shooting alogthim function in terms of 

    rho = air density
    
    g = Accl due to gravity
    
    m = mass 
    
    D = diameter 
    
    v0 = speed 
    
    omega = spin (the axis of rotation is perpentidular with the ground)
    
    phi = azimuthal angle (horivantal angle)
    
    Theta = elevation angle (vertical angle)
    
[inspiration can be taken from the baseball homework, hw 7]

Second, set up enviroemnts with proper dimensions for the two different types of kicks, including the goal and its target area. 
    1. free kick 
    2. corner kick

Third, run shooting alogthim through range of paramters. 

In [9]:
rho = 1.2     # kg m^-3
g= 9.81     # kg m s^-2
m = 0.436    # kg
D = 0.218     # m
r = D/2
v0 = np.arange(24, 30.1, .1)   # m s^-1
omega = np.arange(0, 75.6, .2)   # rad s^-1

In [14]:
#parameters for free kick
fphis = np.arange(-0.0349066, 0.0872665, .001)  # rad [-2 deg to 5 deg]
fthetas = np.arange(.174533, 0.349066, .001) # rad [10 deg to 20 deg]

In [15]:
#parameters for corner kick
cphis = np.arange(-0.0698132, 0.0698132, .001)  # rad [-4 deg to 4 deg]
cthetas = np.arange(0.349066, 0.523599, .001) # rad [20 deg to 30 deg]

Objectives
 1. talk about drag and lift coeffcients of ball
 2. simulate trajectories of free kick and corner kick
 3. plot in 2D and 3D and desricbe paramters of trajectories
 4. Talk in depth about at one goal scoring shot for each: include parameters plots and a plot where omega = 0 to show influence of spin.

trajectory of free kick should stop calculating when
  1. ball reaches goal line
  2. ball touches ground
  3. ball touches wall

trajectory of corner kick should stop calculating when
  1. ball reaches goal line
  2. ball touches ground

Possbily use 3D-plotting to show trajectories of shots. (especally goal-making shots). guidense can come from 14_PDEs_2.ipynb

include refences and acknolegdments 


In [16]:
vcrit = 12.19
vs = 1.309

In [51]:
def F_G(m):
    return -m*g

def C_D(v, S):
    if np.linalg.norm(v) < np.linalg.norm(vcrit) or S < .005:
        return .155 + .346/(1+np.e**((v-vcrit)/vs))
    else:
        return .4127*S**.3056

def F_D(v, S):
    return -1/2*C_D(v, S)*rho*A*np.linalg.norm(v)*v

def C_L(S):
    return 1/2*S**.4

def F_M(omega, v, S):
    return 1/2*C_L(S)*rho*A*np.linalg.norm(v)/np.linalg.norm(omega)*np.cross(omega, v)

In [52]:
def simulate_soccerball(v0, omega, theta, phi, r0, goalpos=np.array([0,0,0]),
                        goalwidth=7.32, goalheight=2.44,
                        h=0.0, g=9.81, rho=1.2,
                        r=0.109, m=0.436, stupidwall=False, wallpos=np.array([-9.16, 0, 0]),
                        wallheight=1.83):
    """simulate soccerball pitch

    Parameters
    ----------
    v0 : array
         initial velocity (vx, vy, vz) in m/s
    omega : array
         angular velocity vector of the ball ("spin"), in rad/s
    theta : float
         horizontal angle ball is kicked at
    phi : float
         vertical angle ball is kicked at
    r0 : array
         initial position of the ball (in m) when it leaves
         as (x, y)
    goalpos : array, optional
         position of goal
    goalwidth : float, optional
         width of goal
    goalheight : float, optional
         height of goal
    h : float, optional
         integration time step in s, default is 0.01 s
    g : float, optional
         acceleration due to gravity, default 9.81 kg/(m*s^2)
    rho : float, optional
         density of air, default 1.225 kg/m^3
    r : float, optional
         radius of the soccerball
    m : float, optional
         mass of the soccerball
    stupidwall : boolean
         if there is a wall 3 m wide not centered with the goal because logic?
    wallpos : array
         position of maybe wall

    Returns
    -------

    positions : array
         The array contains an entry (time, x, y, z) for each time step.
    """
    # all SI units (kg, m)

    goal=False

    omegamag = np.linalg.norm(omega)
    A = np.pi*r**2
    a_G = np.array([0, -g, 0])

    def f(t, y):
        # y = [x, y, z, vx, vy, vz]
        v = y[3:]
        vmag = np.linalg.norm(v)
        S = r*omegamag/vmag
        a_M = F_M(omega, v, S)/m
        a_D = F_D(v, S)/m
        a = a_G + a_D + a_M
        return np.array([y[3], y[4], y[5],
                         a[0], a[1], a[2]])

    x0, y0, z0 = r0
    vx, vy, vz = v0
    t = 0
    positions = []
    # initialize 3D!
    y = np.array([x0, y0, z0, vx, vy, vz], dtype=np.float64)
    if not stupidwall:
        while y[0] < goalpos[0] and y[1] > 0:
            positions.append([t, y[0], y[1], y[2]])  # record t, x and y, z
            y[:] = ode.rk4(y, f, t, h)
            t += h

    elif stupidwall:
        while y[0] < goalpos[0] and y[2] >= 0 and not (y[0] == wallpos[0] and y[1] > wallpos[1]-1 and y[1] < wallpos[1]+2 and y[2] > wallpos[2] and y[2] < wallpos[2]+wallheight):
            positions.append([t, y[0], y[1], y[2]])  # record t, x and y, z
            y[:] = ode.rk4(y, f, t, h)
            t += h

    if y[1] > goalpos[1]-goalwidth/2 and y[1] < goalpos[1]+goalwidth/2 and y[2] < goalpos[2]+goalheight:
        goal = True

    return np.array(positions), goal

In [ ]:
positions, goal = simulate_soccerball(np.array([27, 0, 0]), np.array([0, 0, 30]), .5, 0, (-18.3, 0, 0), goalpos=np.array([0,0,0]),
                        goalwidth=7.32, goalheight=2.44, stupidwall=True)

In [ ]:
print(goal)

In [ ]:
print(positions)

In [13]:
# add your code here; you can copy and paste and modify
# simulate_soccerball() from above

def simulate_soccerball_avmaganced(v0, omega0, r0=None,
                               h=0.01, g=9.81, rho=1.225,
                               r=0.07468/2, m=0.14883,
                               R_homeplate=18.4):
    if r0 is None:
        r0 = np.array([0, 2])  # pitching at 2m height

    omega = np.asarray(omega0)
    omegamag = np.linalg.norm(omega)
    A = np.pi*r**2
    rhoArm = rho*A*r/m

    a_gravity = np.array([0, -g, 0])
    def f(t, y):
        # y = [x, y, z, vx, vy, vz]
        chi = 1/4*(np.sqrt(sum(y[3:]**2))-34)
        if chi<0:
            C_D = .36+.14/(1+np.e**chi)-.27*np.e**(-chi**2)
        else:
            C_D = .36+.14/(1+np.e**chi)-.27*np.e**(-chi**2/4)
        b2 = .5*C_D*rho*A
        v = y[3:]
        vmag = np.linalg.norm(v)
        S = r*omegamag/vmag
        a_magnus = 0.5 * C_L(S) * rhoArm / S * np.cross(omega, v)
        a_drag = -b2/m * vmag * v
        a = a_gravity + a_drag + a_magnus
        return np.array([y[3], y[4], y[5],
                         a[0], a[1], a[2]])
    x0, y0 = r0
    vx, vy = v0
    t = 0
    positions = []
    # initialize 3D!
    y = np.array([x0, y0, 0, vx, vy, 0], dtype=np.float64)

    while y[0] < R_homeplate and y[1] >= 0.2:
        positions.append([t, y[0], y[1], y[2]])  # record t, x and y, z
        y[:] = ode.rk4(y, f, t, h)
        omega = omega * np.e**(-t/5)
        t += h
    return np.array(positions)